![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FMLOps%2FModel+Monitoring&file=bqml-model-monitoring-tutorial.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/MLOps/Model%20Monitoring/bqml-model-monitoring-tutorial.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FMLOps%2FModel%2520Monitoring%2Fbqml-model-monitoring-tutorial.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/MLOps/Model%20Monitoring/bqml-model-monitoring-tutorial.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/MLOps/Model%20Monitoring/bqml-model-monitoring-tutorial.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# BigQuery ML (BQML) - Model Monitoring Tutorial 
---
**UPDATES:**
- **Now With Vertex AI Model Monitoring Integration For [Monitoring visualization](https://cloud.google.com/bigquery/docs/model-monitoring-overview#monitoring_visualization)** - March 2025
---

This notebook provides an in-depth look at model monitoring with BigQuery ML that accompanies the [blog announcing these features](https://cloud.google.com/blog/products/data-analytics/monitor-ml-model-skew-and-drift-in-bigquery). It complements the introductory notebook, ['BigQuery ML (BQML) - Model Monitoring Introduction'](./bqml-model-monitoring-introduction.ipynb), and delves deeper into various monitoring techniques and tools.

**Key Areas Covered:**

* **Data Preparation and Understanding:**
    * Employing `ML.DESCRIBE_DATA` to analyze feature distributions and identify potential data quality issues.
    * Splitting data into training, evaluation, and test sets while preserving data balance.
* **Model Training and Evaluation:**
    * Training a random forest model using `CREATE MODEL` with the `TRANSFORM` clause for incorporating feature engineering.
    * Evaluating model performance using `ML.EVALUATE` and `ML.CONFUSION_MATRIX` on different data splits.
* **Model Serving and Data Drift Simulation:**
    * Generating predictions with `ML.PREDICT` and simulating data drift over time to represent real-world scenarios.
* **Model Monitoring Techniques:**
    * Implementing `ML.VALIDATE_DATA_SKEW` to detect changes between training and serving data distributions.
    * Using `ML.VALIDATE_DATA_DRIFT` to identify gradual shifts in the serving data distribution over time.
    * Understanding the metrics used for monitoring: L-infinity distance and Jensen-Shannon divergence.
* **TensorFlow Data Validation (TFDV) Integration:**
    * Leveraging `ML.TFDV_DESCRIBE` and `ML.TFDV_VALIDATE` to generate descriptive statistics and detect anomalies.
    * Visualizing feature distributions using `tfdv.visualize_statistics` for comparative analysis.
* **Monitoring Feature Attributions and Drift:**
    * Employing `ML.EXPLAIN_PREDICT` to obtain feature attributions and understand feature importance for predictions.
    * Monitoring drift in feature attributions over time to gain insights into model behavior changes. 
* **Continuous Monitoring and Automation:**
    * Building a BigQuery SQL job using procedural language to automate skew and drift detection.
    * Incorporating alerting mechanisms and model retraining when anomalies are detected.
    * Discussing options for automating monitoring jobs:
        * BigQuery Scheduled Queries
        * Dataform
        * Workflows
        * Cloud Composer
        * Vertex AI Pipelines
* **Model Monitoring for Online Inference with Vertex AI:**
    * Deploying the trained BigQuery ML model to Vertex AI for online predictions. 
    * Enabling request-response logging to BigQuery for continuous monitoring of online predictions.

**Target Audience:**

This tutorial is designed for data scientists, machine learning engineers, and individuals with a basic understanding of BigQuery, SQL, and machine learning concepts who want to delve deeper into model monitoring with BigQuery ML. 


---
## Colab Setup

When running this notebook in [Colab](https://colab.google/) or [Colab Enterprise](https://cloud.google.com/colab/docs/introduction), this section will authenticate to GCP (follow prompts in the popup) and set the current project for the session.

In [ ]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [ ]:
try:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)
    print('Colab authorized to GCP')
except Exception:
    print('Not a Colab Environment')
    pass

---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [ ]:
# tuples of (import name, install name, min_version)
packages = [
    ('tensorflow_data_validation', 'tensorflow-data-validation'),
    ('tensorflow_metadata', 'tensorflow-metadata'),
    ('google.cloud.aiplatform', 'google-cloud-aiplatform'),
    ('google.cloud.bigquery', 'google-cloud-bigquery'),
    ('google.cloud.bigquery_datatransfer', 'google-cloud-bigquery-datatransfer'),
    ('numpy', 'numpy'),
    ('pandas', 'pandas')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### API Enablement

In [ ]:
!gcloud services enable aiplatform.googleapis.com

### Restart Kernel (If Installs Occured)
After a kernel restart the code submission can start with the next cell after this one.

In [ ]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
    IPython.display.display(IPython.display.Markdown("""<div class=\"alert alert-block alert-warning\">
        <b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. The previous cells do not need to be run again⚠️</b>
        </div>"""))

---
## Setup

Packages:

In [ ]:
import json

from IPython.display import Image, display, HTML

Clients:

In [ ]:
%load_ext google.cloud.bigquery

Prepare the code below for your environment.

This notebook takes advantage of the [BigQuery IPython magic](https://cloud.google.com/python/docs/reference/bigquery/latest/magics) for legibility and ease of copy/pasting to BigQuery SQL editor.  If this notebook is being used from an environment that can run notebooks it needs further preparation: Colab, Colab Enterprise, Vertex AI Workbench Instances, or BigQuery Studio with a Python Notebook.  The SQL code in these cells uses the fully qualified [BigQuery table](https://cloud.google.com/bigquery/docs/tables-intro) names in the form `projectname.datasetname.tablename`.  Prepare for your environment by:
- Edit > Find
    - Find: `statmike-mlops-349915`
    - Replace: `<your project id>`
    - Replace All

---
## BigQuery Source Data

This project will uses the following data source as a tutorial.  It is good for examples like this because it is small, has few columns, but contains categorial and numerical data types.

The source table is a BigQuery Public Dataset table.  The following cell uses the [BigQuery IPython magic](https://cloud.google.com/python/docs/reference/bigquery/latest/magics) to retrieve 5 rows of the table for review.  This data is known as [Palmer Penguins](https://allisonhorst.github.io/palmerpenguins/) data: 

```
@Manual{,
  title = {palmerpenguins: Palmer Archipelago (Antarctica) penguin data},
  author = {Allison Marie Horst and Alison Presmanes Hill and Kristen B Gorman},
  year = {2020},
  note = {R package version 0.1.0},
  doi = {10.5281/zenodo.3960218},
  url = {https://allisonhorst.github.io/palmerpenguins/},
}
```


There are 334 observations of 4 numerical features (culman length, culmen depth, flipper length, body mass) and 2  categorical features (island, sex) that represent 3 species of penguins.

In [ ]:
%%bigquery
SELECT *
FROM `bigquery-public-data.ml_datasets.penguins`
LIMIT 5

---
## **Goal: Train, Serve, And Monitor A Model**

**Train A Model**

In this workflow our goal is to train and operationalize a machine leanring model that can classify unidentified penguins into the correct `species` using only measurements (`culmen_length_mm`, `culmen_depth_mm`, `flipper_length_mm`, `body_mass_g`), location (`island`) and gender (`sex`).  That means serving predictions, monitoring features, and potentially retraining the model.

**Operationalize The Model**

Taking a trained model to production introduces challenges. The model needs to be regularly evaluated to understand it's performance once put into production. This can be challenging as actual values used to assess the accuracy of predictions may arrive later than the predictions. Continuous monitoring bridges this gap with early signals in two key ways. First, monitoring the distributions of each of the models features for shift (called **skew**) from the training data to the current data. Then, also monitoring for any gradual **drift** in the distribution over time. Setting thresholds and getting alerts for features on each of this distribution changes gives early signals that the model needs to be carefully considered for evaluation and re-training to ensure effectiveness. If the evaluation can be automated then this can even extend to **continuous training**. 

---
## Data Preparation

### Understand The Data Source With `ML.DESCRIBE_DATA`

Reviewing a few records, like above, gives a good sense of how the data is arranged. Before proceeding with machine learning techniques it is important to understand more about these raw columns.  Are they ready to use a features in a model or is some form of feature engineering needed first?  For this, the distribution of values is an important starting point.  

While SQL could be used to look at the distribution, it would be a time consuming process and requires different techniques for different data types like numerical, string, boolean, dates, times, array and struct version of these, and arrays of structs.

To make this process fast and simple, the new [`ML.DESCRIBE_DATA`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-describe-data)

function is used to get a single row for each column the describes the data distribution:

In [ ]:
%%bigquery
SELECT *
FROM ML.DESCRIBE_DATA(
  TABLE `bigquery-public-data.ml_datasets.penguins`
)

Some observations:
- All columns have `num_rows` and `num_nulls`
- Numerical columns have `min`, `max`, `stddev`, `median`, and `quantiles` showing the distribution of the data in the column
- Categorical (string) columns have `min`, `max` values representing the class levels and a `unique` column show how many class levels

Data quality observation:
- The column `sex` has both missing values (10 indicated in `num_nulls`) as well as a `min` values of `.`.  This appears to be a data quality issue that needs fixing.

### Create A BigQuery Dataset

Create a new [BigQuery Dataset](https://cloud.google.com/bigquery/docs/datasets) as a working location for this workflow:

In [ ]:
%%bigquery
CREATE SCHEMA IF NOT EXISTS `statmike-mlops-349915.bqml_model_monitoring`
    OPTIONS(
        location = 'US'
    )

### Prepare The Source Data

Make a copy of the table source in the new BigQuery dataset with fixes applied to the data quality issue identified for the `sex` column with values of `.`.

> Note: A copy is being made in this case because the source project is `bigquery-public` which is not editable.

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `statmike-mlops-349915.bqml_model_monitoring.training` AS
    SELECT * EXCEPT(sex),
        CASE WHEN sex = '.' THEN NULL ELSE sex END AS sex
    FROM `bigquery-public-data.ml_datasets.penguins`

### Split The Data

Depending on the ML technique, it may be desired to split the data into partitions for training, evaluation, and testing (in this case monitoring examples). 

The following cell creates a version with a new column column named `splits` with values [`TRAIN`, `EVAL`, `TEST`].  The data is first grouped (stratified) by the values of `species` and `island` to preserve any imbalance across the groups. 

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `statmike-mlops-349915.bqml_model_monitoring.training_split` AS
    WITH
        # randomized numbering within groups (species, island)
        RANDOM AS (
            SELECT *,
                ROW_NUMBER() OVER (PARTITION BY species, island ORDER BY RAND()) AS sequence
            FROM `statmike-mlops-349915.bqml_model_monitoring.training`
        ),
        # get group sizes
        GROUP_SIZES AS (
            SELECT species, island, COUNT(*) AS count
            FROM `statmike-mlops-349915.bqml_model_monitoring.training`
            GROUP BY species, island
        )
    SELECT
        * EXCEPT(sequence, count),
        CASE
            WHEN sequence <= CEIL(.2 * count) AND species is not Null THEN 'TEST'
            WHEN sequence <= CEIL(.3 * count) THEN 'EVAL'
            ELSE 'TRAIN'
        END AS splits
    FROM RANDOM
    LEFT OUTER JOIN GROUP_SIZES USING(species, island)

Review the data by `splits`:

In [ ]:
%%bigquery
SELECT species, island,
    COUNT(*) as count,
    100 * COUNTIF(splits = 'TRAIN')/COUNT(*) AS TRAIN_PCT,
    100 * COUNTIF(splits = 'EVAL')/COUNT(*) AS EVAL_PCT,
    100 * COUNTIF(splits = 'TEST')/COUNT(*) AS TEST_PCT
FROM `statmike-mlops-349915.bqml_model_monitoring.training_split`
GROUP BY species, island

### Review the Training Split With: `ML.DESCRIBE_DATA`

Use [`ML.DESCRIBE_DATA`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-describe-data) to first review the training split of the source data.  This time, some of the additional parameters of the function are useful:
- `top_k`: get the top 3 most frequent categories for string columns (default = 1)
- `num_quantiles`: get 4 quantiles for numerical columns (default = 2)

In [ ]:
%%bigquery
SELECT *
FROM ML.DESCRIBE_DATA(
    (
        SELECT * EXCEPT(splits)
        FROM `statmike-mlops-349915.bqml_model_monitoring.training_split`
        WHERE splits = 'TRAIN'
    ),
    STRUCT(3 AS top_k, 4 AS num_quantiles)
)

---
## Model Training

Create a model trained to classify `species` for the traning records.  Here, directly in BigQuery, the [`CREATE MODEL` statement for random forest](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create-random-forest) is used.  This uses the `TRANSFORM` clause within the model to embed user specified transfomations within the model for use during serving and model monitoring which will both be show later in this introduction.

From reviewing the training data above, the following transformations will be tried:
- Apply [`ML.ROBUST_SCALER`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-robust-scaler) to `body_mass_g`
- Apply [`ML.STANDARD_SCALER`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-standard-scaler) to `culmen_depth_mm` and `culmen_length_mm`
- Apply [`ML.QUANTILE_BUCKATIZE`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-quantile-bucketize) to `flipper_length_mm`
- Apply [`ML.IMPUTER`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-imputer) to `island`, and `sex`

In [ ]:
%%bigquery
CREATE OR REPLACE MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`
    TRANSFORM(
        ML.ROBUST_SCALER(body_mass_g) OVER() AS body_mass_g,
        ML.STANDARD_SCALER(culmen_length_mm) OVER() AS culmen_length_mm,
        ML.STANDARD_SCALER(culmen_depth_mm) OVER() AS culmen_depth_mm,
        ML.QUANTILE_BUCKETIZE(flipper_length_mm, 3) OVER() AS flipper_length_mm,
        ML.IMPUTER(sex, 'most_frequent') OVER() AS sex,
        ML.IMPUTER(island, 'most_frequent') OVER() AS island,
        species, split
    )
    OPTIONS(
        MODEL_TYPE = 'RANDOM_FOREST_CLASSIFIER',
        INPUT_LABEL_COLS = ['species'],
        
        # data specifics
        DATA_SPLIT_METHOD = 'CUSTOM',
        DATA_SPLIT_COL = 'split',
        
        # model specifics
        AUTO_CLASS_WEIGHTS = FALSE,
        NUM_PARALLEL_TREE= 150,
        TREE_METHOD = 'HIST',
        SUBSAMPLE = 0.85,
        COLSAMPLE_BYTREE = 0.9,
        ENABLE_GLOBAL_EXPLAIN = TRUE,
        
        # register model in Vertex AI For Online Serving
        MODEL_REGISTRY = 'VERTEX_AI',
        VERTEX_AI_MODEL_ID = 'classify_species_rf'
    )
AS
    SELECT * EXCEPT(splits),
        CASE WHEN splits = 'TRAIN' THEN FALSE
        ELSE TRUE END AS split
    FROM `statmike-mlops-349915.bqml_model_monitoring.training_split`
    WHERE splits != 'TEST'

In [ ]:
%%bigquery
SELECT *
FROM ML.TRAINING_INFO(MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`)
ORDER BY iteration

### Evaluation With `ML.EVALUATE`

BigQuery ML models can be directly evaluated with the [`ML.EVALUATE` function](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-evaluate).  The evaluation of the model on evaluation data specified during training can be done by just specifying the model.  To evaluate the model on new data, like a `TEST` split or even the `TRAIN` split, the records can be specified with a query statement as seen in the following evaluations.

> An evaluation split with 10% of records was created above.  This set would normally be used for tuning model hyperparameters, like `BATCH_SIZE`, `DROPOUT`, `HIDDEN_UNITS`, `LEARN_RATE`, and/or `OPTIMIZER` in the case of the DNN used here.  This could also be done directly in the BigQuery `CREATE MODEL` statement with [hyperparameter tuning](https://cloud.google.com/bigquery/docs/hp-tuning-overview).

In [ ]:
%%bigquery
SELECT *
FROM ML.EVALUATE (
    MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`
)

### Evaluate The Model: Training Data Split (`TRAIN`)

Evaluating the model on the training data is good for setting a baseline for the likely best possible performance of the model.  

In [ ]:
%%bigquery
SELECT *
FROM ML.EVALUATE (
    MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`,
    (
        SELECT *
        FROM `statmike-mlops-349915.bqml_model_monitoring.training_split`
        WHERE splits = 'TRAIN'
    )
)

### Evaluate The Model: `TEST` Data Split

Evaluating the model on the `TEST` split is a good gauge of performance in general as it represents data the model was not exposed to during training: 

In [ ]:
%%bigquery
SELECT *
FROM ML.EVALUATE (
    MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`,
    (
        SELECT *
        FROM `statmike-mlops-349915.bqml_model_monitoring.training_split`
        WHERE splits = 'TEST'
    )
)

### Evaluate The Classification Performance with `ML.CONFUSION_MATRIX`

This model is a classification model and reviewing the true versus predicted classification categories can be done with the [`ML.CONFUSION_MATRIX` function](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-confusion).

In [ ]:
%%bigquery
SELECT *
FROM ML.CONFUSION_MATRIX (
    MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`
)

In [ ]:
%%bigquery
SELECT *
FROM ML.CONFUSION_MATRIX (
    MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`,
    (
        SELECT *
        FROM `statmike-mlops-349915.bqml_model_monitoring.training_split`
        WHERE splits = 'TRAIN'
    )
)

In [ ]:
%%bigquery
SELECT *
FROM ML.CONFUSION_MATRIX (
    MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`,
    (
        SELECT *
        FROM `statmike-mlops-349915.bqml_model_monitoring.training_split`
        WHERE splits = 'TEST'
    )
)

---
## Model Serving

Once a version of the model is trusted to serve predictions we can put it into use.  At first, this means creating and using predictions from the model.  This is quickly followed by important considerations:

>Has the system that is being measured, that creates the data, or that records the data changed in a way that could impact the models ability to correctly predict/classify?

It seems like a good idea to continously run evaluations to make sure the model accuracy does not degrade - and it is.  There can be a lag between new data and knowing the actual outcome which makes using using evaluations to monitor models a potentially reactive, or late signal.  To overcome this we can also monitor the model.  More on this later on below.

### Predictions With `ML.PREDICT`

Use [`ML.PREDICT`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-predict) to serve predictions with the model directly in BigQuery.

In [ ]:
%%bigquery
SELECT *
FROM ML.PREDICT(
    MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`,
    (
        SELECT *
        FROM `statmike-mlops-349915.bqml_model_monitoring.training_split`
        LIMIT 5
    )
)

---
## Serving Data

The data source for training and evaluation represents what is known about a system at a point-in-time.  In real world systems measurements can change for many reasons, for example:
- training/serving skew due to measurement and data systems:
    - the tool used for measurement may change resulting is slight shifts and variations
    - the system reporting results might be updated and introduce errors like rearranging the order or parameters and format of the results
    - the data processing systems might have changes that alter the calculations of measurements
- the system, environment, or subjects being measured may:
    - change behaviors
    - short-term even forces change
    - have environmental changes that impact them
    - have seasonal changes that were not represented in the training data
    - change in general due to upstream causes

### Introduce Changes Over Time

To illustrate the impact of change over time, the following code creates a new set of data from the test split.  It creates a series of changes to measurements for each of the past 30 days.  In each case, the values of the parameters `flipper_length_mm` and `body_mass_g` are allowed to slightly increase which can accumulate over the timespan. This might mimic a potential growth season for the penguins that might not have been represented in the training data.

This table of serving data could be sourced form where your features are coming from.  In the case of an online model, like [Vertex AI Prediction Endpoints](https://cloud.google.com/vertex-ai/docs/predictions/overview), then [prediction request could be logged to BigQuery](https://cloud.google.com/vertex-ai/docs/predictions/online-prediction-logging#enabling-and-disabling) for use in the model monitoring tasks below.  This online serving pattern is covered later in this tutorial.

In [ ]:
%%bigquery
DECLARE counter INT64 DEFAULT 30;

# actual data as starting point - add entity_id, and feature_timestamp:
CREATE OR REPLACE TABLE `statmike-mlops-349915.bqml_model_monitoring.serving` AS
    SELECT * EXCEPT(flipper_length_mm, body_mass_g),
    TIMESTAMP_SUB(TIMESTAMP(CURRENT_DATE()), INTERVAL counter DAY) AS instance_timestamp,
    flipper_length_mm as flipper_length_mm,
    body_mass_g as body_mass_g
    FROM `statmike-mlops-349915.bqml_model_monitoring.training`
    WHERE RAND() <= 0.20;

LOOP
    SET counter = counter - 1;
    IF counter < 0 THEN LEAVE;
    END IF;
    INSERT `statmike-mlops-349915.bqml_model_monitoring.serving`
        SELECT * EXCEPT(instance_timestamp, flipper_length_mm, body_mass_g),
            TIMESTAMP_ADD(instance_timestamp, INTERVAL 1 DAY) AS instance_timestamp,
            CASE
                WHEN RAND() >= 0.75 THEN flipper_length_mm
                ELSE flipper_length_mm + 6*(RAND()-0.1)
            END AS flipper_length_mm,
            CASE
                WHEN RAND() >= 0.75 THEN body_mass_g
                ELSE body_mass_g + 120*(RAND()-0.1)
            END AS body_mass_g,
        FROM `statmike-mlops-349915.bqml_model_monitoring.serving`
        WHERE instance_timestamp >= (SELECT MAX(instance_timestamp) FROM `statmike-mlops-349915.bqml_model_monitoring.serving`);
END LOOP;

In [ ]:
%%bigquery serve
SELECT
    DATE(instance_timestamp) AS DATE,
    AVG(flipper_length_mm) AS flipper_length_mm,
    AVG(body_mass_g) AS body_mass_g
FROM `statmike-mlops-349915.bqml_model_monitoring.serving`
GROUP BY DATE
ORDER BY DATE

In [ ]:
serve.set_index('DATE', inplace = True)

In [ ]:
#serve

In [ ]:
serve['flipper_length_mm'].plot(legend = True, figsize = (8, 4))

In [ ]:
serve['body_mass_g'].plot(legend = True, figsize = (8, 4))

---
## Monitoring With Evaluations

It might seem like ongoing model evaluations are a great solution to model monitoring.  If the actual known values are available immediately then this could be possible to use for monitoring.  However, detecting a change in evaluation metrics means the model has realized a degradation.  It is a lagging signal.  To be more proactive we should monitor the inputs to the model, the features.  More on that in the next section.

First, look at the evaluation metrics for the most recent days data which has the cumulative change of the past 30 days introduced above.  In this case, the actual known values is known. This may not be possible in many real-world examples where the actual known values are:
- delayed by minutes, hours, days, or even longer
- not always known for all instances
- potentially in a different status that can change over a period of time before being confirmed as final

In reviewing the evaluation for the most recent instances here, the `precision` has dropped and the `log_loss` has increased drastically.  The confusion matrix shows Chinstrap penguins's are now being misclassified much more often than when the model was trained.  

The question becomes "How soon could this have been noticed?".

In [ ]:
%%bigquery
SELECT *
FROM ML.EVALUATE (
    MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`,
    (
        SELECT *
        FROM `statmike-mlops-349915.bqml_model_monitoring.serving`
        WHERE DATE(instance_timestamp) >= CURRENT_DATE()
    )
)

In [ ]:
%%bigquery
SELECT *
FROM ML.CONFUSION_MATRIX (
    MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`,
    (
        SELECT *
        FROM `statmike-mlops-349915.bqml_model_monitoring.serving`
        WHERE DATE(instance_timestamp) >= CURRENT_DATE()
    )
)

### Evaluations: Over Time

In this case, we know the actual identification of the `species` and are measuring the same penguins daily.  Reviewing the evaluations daily could give a good sense of how the model is performing on a daily basis.

The result of the daily evaluations below shows that the log_loss starts to increase immediately and it takes more than a week for a noticable impact on precision/recall/accuracy/f1_score. 

What if the actual values where only known after a lag of days or weeks?  How quickly would evaluations be able to indicate the model's accuracy is possibly questionable?  The answer is a different approach - Model Monitoring.  This is covered in the section that follows.

In [ ]:
%%bigquery evaluations
DECLARE counter INT64 DEFAULT 30;
DECLARE result STRUCT<precision FLOAT64, recall FLOAT64, accuracy FLOAT64, f1_score FLOAT64, log_loss FLOAT64, roc_auc FLOAT64, eval_date DATE>;
DECLARE results ARRAY<STRUCT<precision FLOAT64, recall FLOAT64, accuracy FLOAT64, f1_score FLOAT64, log_loss FLOAT64, roc_auc FLOAT64, eval_date DATE>> DEFAULT [];
LOOP
    IF counter < 0 THEN LEAVE;
    END IF;
    SET result = (
            SELECT AS STRUCT precision, recall, accuracy, f1_score, log_loss, roc_auc, DATE_SUB(CURRENT_DATE(), INTERVAL counter DAY) AS eval_date
            FROM ML.EVALUATE (
                MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`,
                (
                    SELECT *
                    FROM `statmike-mlops-349915.bqml_model_monitoring.serving`
                    WHERE DATE(instance_timestamp) = DATE_SUB(CURRENT_DATE(), INTERVAL counter DAY)
                )
            )
    );
    SET results = ARRAY_CONCAT(results,[result]);
    SET counter = counter - 1;
END LOOP;
SELECT * FROM UNNEST(results);

In [ ]:
evaluations

In [ ]:
evaluations.set_index('eval_date', inplace = True)

In [ ]:
evaluations.plot(y = ['log_loss', 'f1_score', 'precision'], legend = True, title = 'Evaluation Metrics Over Last 30 days', figsize = (8, 3), grid = True)

---
## **Model Monitoring**

Reviewing model evaluations over time does show a pattern of deterioration in the example above.  But at what point is it actionable?  And what if each days evaluations are relying on actual values that took hours, days, weeks, or even months to be available?  

Model monitoring takes a different approach.  It goes back to the source of the model, each input feature.  These features form a distribution at the input to the model (the training data) and this can be used to compare to over time.  By setting thresholds on these comparisons to the original training data, alerts can be used to notify that a models accuracy *could be* in question.  This comparison to the training data is called **skew** detection.

Similarly, each days values could be compared to the previous days, or weeks values.  These comparisons are referred to as **drift** detection.  

Both of these types of comparisons can be done directly in BigQuery with two functions built for these tasks:
- [`ML.VALIDATE_DATA_SKEW`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-validate-data-skew)
- [`ML.VALIDATE_DATA_DRIFT`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-validate-data-drift)

### Monitoring Skew (Change from training) With `ML.VALIDATE_DATA_SKEW`

Compare today, or any data, to the training data by directly comparing to the the model created in BigQuery.

>**NOTE:** By adding the `enable_visualization_link` argument to the `ML.VALIDATE_DATA_SKEW` function it adds the Vertex AI Model Monitoring console link in the `visualization_link` column of the output.  Read more about [Monitoring visualization](https://cloud.google.com/bigquery/docs/model-monitoring-overview#monitoring_visualization) in the BigQuery ML [Model monitoring overview](https://cloud.google.com/bigquery/docs/model-monitoring-overview).

In [ ]:
%%bigquery skew
SELECT *
FROM ML.VALIDATE_DATA_SKEW(
    # base
    MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`,
    (# compare
        SELECT *
        FROM `statmike-mlops-349915.bqml_model_monitoring.serving`
        WHERE DATE(instance_timestamp) >= CURRENT_DATE()
    )
    ,STRUCT(TRUE AS enable_visualization_link)
);

In [ ]:
skew

In [ ]:
display(HTML(f'<a href="{skew.visualization_link[0]}" target="_blank" style="font-size: 18px;">Open Google Cloud Console in a new tab</a>'))

In [ ]:
display(HTML('<div style="text-align: center;"><img src="./resources/tutorial_skew.gif" style="border: 2px solid black; width: auto; height: 600px;"></div>'))

### Monitoring Drift (Change Over Time) With `ML.VALIDATE_DATA_DRIFT`

Compare today to 5 days ago and set the a lower, more sensitive, threshold of `0.03` (default is `0.3`) for all features.


>**NOTE:** By adding the `MODEL` to the `ML.VALIDATE_DATA_DRIFT` function it adds the Vertex AI Model Monitoring console link in the `visualization_link` column of the output.  Read more about [Monitoring visualization](https://cloud.google.com/bigquery/docs/model-monitoring-overview#monitoring_visualization) in the BigQuery ML [Model monitoring overview](https://cloud.google.com/bigquery/docs/model-monitoring-overview).

In [ ]:
%%bigquery drift
SELECT *
FROM ML.VALIDATE_DATA_DRIFT(
    (# base
        SELECT * EXCEPT(instance_timestamp, species)
        FROM `statmike-mlops-349915.bqml_model_monitoring.serving`
        WHERE DATE(instance_timestamp) = DATE_SUB(CURRENT_DATE(), INTERVAL 5 DAY)
    ),
    (# compare
        SELECT * EXCEPT(instance_timestamp, species)
        FROM `statmike-mlops-349915.bqml_model_monitoring.serving`
        WHERE DATE(instance_timestamp) = CURRENT_DATE() 
    ),
    STRUCT(
        0.03 AS categorical_default_threshold,
        0.03 AS numerical_default_threshold
    )
    , MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`
);

In [ ]:
drift

In [ ]:
display(HTML(f'<a href="{drift.visualization_link[0]}" target="_blank" style="font-size: 18px;">Open Google Cloud Console in a new tab</a>'))

In [ ]:
display(HTML('<div style="text-align: center;"><img src="./resources/tutorial_drift.gif" style="border: 2px solid black; width: auto; height: 600px;"></div>'))

### Monitoring Skew (Manually) - As Drift from Training Data

Since the `ML.VALIDATE_DATA_DRIFT` is comparing two tables, it could also be used for **skew** by comparing the training data to new data, like todays data:

In [ ]:
%%bigquery skew_manual
SELECT *
FROM ML.VALIDATE_DATA_DRIFT(
    (# base
        SELECT *
        FROM `statmike-mlops-349915.bqml_model_monitoring.training_split`
        WHERE splits = 'TRAIN'
    ),
    (# compare
        SELECT * EXCEPT(instance_timestamp, species)
        FROM `statmike-mlops-349915.bqml_model_monitoring.serving`
        WHERE DATE(instance_timestamp) = CURRENT_DATE() 
    ),
    STRUCT()
    , MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`
);

In [ ]:
skew_manual

In [ ]:
display(HTML(f'<a href="{skew_manual.visualization_link[0]}" target="_blank" style="font-size: 18px;">Open Google Cloud Console in a new tab</a>'))

In [ ]:
display(HTML('<div style="text-align: center;"><img src="./resources/tutorial_skew_manual.gif" style="border: 2px solid black; width: auto; height: 600px;"></div>'))

---
### TensorFlow Data Validation (TFDV) In BigQuery

BigQuery offers two functions that bring the power of the TensorFlow's [TFX platform](https://www.tensorflow.org/tfx) with built-in [TensorFlow Data Validation (TFDV)](https://www.tensorflow.org/tfx/data_validation/get_started) module directly inside of BigQuery.
- [`ML.TFDV_DESCRIBE`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-tfdv-describe) to generate descriptive statistics of any table
- [`ML.TFDV_VALIDATE`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-tfdv-validate) to compare statistics of two tables

#### Generate Statistics with `ML.TFDV_DESCRIBE`

Generate statistics for any table, in this case the training data features or a timespan of serving data features.  This is like using the TensorFlow's [TFX platform](https://www.tensorflow.org/tfx) with built-in [TensorFlow Data Validation (TFDV)](https://www.tensorflow.org/tfx/data_validation/get_started) module's [`tfdv.generate_statistics_from_csv()`](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/generate_statistics_from_csv) method.

The descriptions can be used in comparisons for skew or drift detection with the `ML.TFDV_VALIDATE` function, covered next, as well as in the visualization tool `tfdv.visualize_statistics` also covered below.

In [ ]:
%%bigquery train_describe
SELECT *
FROM ML.TFDV_DESCRIBE(
    (
        SELECT * EXCEPT(splits, species)
        FROM `statmike-mlops-349915.bqml_model_monitoring.training_split`
        WHERE splits = 'TRAIN'
    )
)

In [ ]:
train_describe = json.loads(train_describe['dataset_feature_statistics_list'].iloc[0])

In [ ]:
train_describe['datasets'][0]['features'][0]

In [ ]:
%%bigquery today_describe
SELECT *
FROM ML.TFDV_DESCRIBE(
    (
        SELECT * EXCEPT(instance_timestamp, species)
        FROM `statmike-mlops-349915.bqml_model_monitoring.serving`
        WHERE DATE(instance_timestamp) = CURRENT_DATE()
    )
)

In [ ]:
today_describe = json.loads(today_describe['dataset_feature_statistics_list'].iloc[0])

In [ ]:
#today_describe['datasets'][0]['features'][0]

#### Visualize Statistics with `tfdv.visualize_statistics`

Use the [`tfdv.visualize_statistics`](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/visualize_statistics) tool to visualize the generated descriptions and directly compare them.  This presents visuals using [Facets Overview](https://pair-code.github.io/facets/).

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow_data_validation as tfdv
import tensorflow_metadata as tfmd
from google.protobuf import json_format

In [ ]:
tfdv.visualize_statistics(
    lhs_statistics = json_format.ParseDict(train_describe, tfmd.proto.statistics_pb2.DatasetFeatureStatisticsList()),
    rhs_statistics = json_format.ParseDict(today_describe, tfmd.proto.statistics_pb2.DatasetFeatureStatisticsList()),
    lhs_name = 'Training Data Stats',
    rhs_name = 'Serving Data Stats - For Today'
)

In [ ]:
display(HTML('<div style="text-align: center;"><img src="./resources/tutorial_tfdv_visualize.gif" style="border: 2px solid black; width: auto; height: 400px;"></div>'))

#### Compare And Detect Anomalies with `ML.TFDV_VALIDATE`

Compare the distribution of any table to a base table and compute distance statistics to detect anomalies.  This is like using the TensorFlow's [TFX platform](https://www.tensorflow.org/tfx) with built-in [TensorFlow Data Validation (TFDV)](https://www.tensorflow.org/tfx/data_validation/get_started) module's [validate_statistics api](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/validate_statistics).

These comparison can be made to training data for **skew** detection or previous serving data for **drift** detection.

In [ ]:
%%bigquery validate
WITH
    TRAIN AS (
        SELECT * EXCEPT(splits, species)
        FROM `statmike-mlops-349915.bqml_model_monitoring.training_split`
        WHERE splits = 'TRAIN'
    ),
    SERVE AS (
        SELECT * EXCEPT(instance_timestamp, species)
        FROM `statmike-mlops-349915.bqml_model_monitoring.serving`
        WHERE DATE(instance_timestamp) = CURRENT_DATE()
    )
SELECT ML.TFDV_VALIDATE(
    (SELECT * FROM ML.TFDV_DESCRIBE(TABLE TRAIN)),
    (SELECT * FROM ML.TFDV_DESCRIBE(TABLE SERVE)),
    'SKEW'
) as validate

In [ ]:
validate = json.loads(validate['validate'].iloc[0])

In [ ]:
#validate['anomaly_info']

#### Display Anomalies with `tfdv.display_anomalies`

Use the [`tfdv.display_anomalies`](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/display_anomalies) tool to display a table of anomaly detections with descriptions.

In [ ]:
if 'generate_legacy_feature_spec' in validate['baseline'].keys():
    del validate['baseline']['generate_legacy_feature_spec']

In [ ]:
tfdv.display_anomalies(
    anomalies = json_format.ParseDict(validate, tfmd.proto.anomalies_pb2.Anomalies())
)

---
## Understanding Monitoring Thresholds

The `ML.VALIDATE_DATA_DRIFT`, `ML.VALIDATE_DATA_SKEW`, and `ML.TFDV_VALIDATE` functions use distance scores to quantify the change in the statistical distribution of features.  The methods of calculating the distance scores are:
- [L-infinity distance](https://en.wikipedia.org/wiki/Chebyshev_distance) for categorical features
- [Jensen-Shannon divergence](https://en.wikipedia.org/wiki/Jensen%E2%80%93Shannon_divergence) for numeric features.  Can also be use for categorical features.

These methods are the same ones used by [TensorFlow Data Validation](https://www.tensorflow.org/tfx/guide/tfdv#training-serving_skew_detection) which is also used in [Vertex AI Model Monitoring](https://cloud.google.com/vertex-ai/docs/model-monitoring/overview#calculating-skew-and-drift).

This section aims to provide intuition on how these two metrics work.  In the data used for this workflow the feature `island` has three possible values: `Biscoe`, `Dream`, and `Torgersen`.  The proportion of data in each of these categories forms a distribution.  This will be used to visually show how both methods of calculating distance scores work.

In [ ]:
import pandas as pd
import numpy as np

### Retrieve Distributions: Training & Serving

Use the `ML.TFDV_DESCRIBE` function to get the statistics for the distribution of the `island` feature at both training and for serving.

In [ ]:
%%bigquery island_train
SELECT *
FROM ML.TFDV_DESCRIBE(
    (
        SELECT island
        FROM `statmike-mlops-349915.bqml_model_monitoring.training_split`
        WHERE splits = 'TRAIN'
    )
)

In [ ]:
%%bigquery island_serve
SELECT *
FROM ML.TFDV_DESCRIBE(
    (
        SELECT island
        FROM `statmike-mlops-349915.bqml_model_monitoring.serving`
    )
)

In [ ]:
island_train = json.loads(island_train['dataset_feature_statistics_list'].iloc[0])
island_serve = json.loads(island_serve['dataset_feature_statistics_list'].iloc[0])

In [ ]:
island_train['datasets'][0]['features'][0]['string_stats']

In [ ]:
island_train_stats = island_train['datasets'][0]['features'][0]['string_stats']['rank_histogram']['buckets']
island_train_stats

In [ ]:
island_serve_stats = island_serve['datasets'][0]['features'][0]['string_stats']['rank_histogram']['buckets']
island_serve_stats

### Combine Distributions In DataFrame

In [ ]:
index = [i['label'] for i in island_train_stats]
train = [i['sample_count'] for i in island_train_stats]
serve = [island_serve_stats[index.index(k)]['sample_count'] for k in [i['label'] for i in island_serve_stats]]
stats = pd.DataFrame(
    {
        'train_n': train,
        'serve_n': serve,
        'train_pct': [x/sum(train) for x in train],
        'serve_pct': [x/sum(serve) for x in serve]
    },
    index = index
)
stats

### Visualize Distributions

In [ ]:
ax = stats[['train_pct', 'serve_pct']].plot.bar(rot=0)
for container in ax.containers:
    ax.bar_label(container)

### L-Infinity Distance

In [ ]:
stats['abs_change_pct'] = abs(stats['train_pct'] - stats['serve_pct'])
stats

In [ ]:
ax = stats['abs_change_pct'].plot.bar()
ax.bar_label(ax.containers[0])

In [ ]:
print(f"The max change in percentage(L-Infinity Distance) is {max(stats['abs_change_pct'])}")

### Jensen-Shannon Divergence

In [ ]:
stats['mix'] = (stats['train_pct'] + stats['serve_pct']) / 2
stats['train_kl'] = stats['train_pct'] * np.log2(stats['train_pct'] / stats['mix'])
stats['serve_kl'] = stats['serve_pct'] * np.log2(stats['serve_pct'] / stats['mix'])
stats['JSD'] = (stats['train_kl'] + stats['serve_kl']) / 2
stats

In [ ]:
print(f"The total for the Jansen-Shannon Divergence is {sum(stats['JSD'])}")

### Verify Calculations with `ML.VALIDATE_DATA_DRIFT`

In [ ]:
%%bigquery
SELECT *
FROM ML.VALIDATE_DATA_DRIFT(
    (
        SELECT island
        FROM `statmike-mlops-349915.bqml_model_monitoring.training_split`
        WHERE splits = 'TRAIN'
    ),
    (
        SELECT island
        FROM `statmike-mlops-349915.bqml_model_monitoring.serving`
    ),
    STRUCT('L_INFTY' AS categorical_metric_type)
    #, MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`
)

In [ ]:
print(f"The max change in percentage(L-Infinity Distance) is {max(stats['abs_change_pct'])}")

In [ ]:
%%bigquery
SELECT *
FROM ML.VALIDATE_DATA_DRIFT(
    (
        SELECT island
        FROM `statmike-mlops-349915.bqml_model_monitoring.training_split`
        WHERE splits = 'TRAIN'
    ),
    (
        SELECT island
        FROM `statmike-mlops-349915.bqml_model_monitoring.serving`
    ),
    STRUCT('JENSEN_SHANNON_DIVERGENCE' AS categorical_metric_type)
    #, MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`
)

In [ ]:
print(f"The total for the Jansen-Shannon Divergence is {sum(stats['JSD'])}")

---
## More Applications For Model Monitoring

---
### Monitoring Transformed Features

The [`CREATE MODEL`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create) statement included a [`TRANSFORM`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create#transform).  This is available to be called directly for input data with the [`ML.TRANSFORM`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-transform) function.  This can be useful for considering monitoring the transformed features.

#### Transformations With `ML.TRANSFORM`

In [ ]:
%%bigquery
SELECT *
FROM ML.TRANSFORM(
    MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`,
    (
        SELECT *
        FROM `statmike-mlops-349915.bqml_model_monitoring.serving`
        LIMIT 5
    )
)

#### Monitoring Skew (Manually) - As Drift from Training Data With `ML.VALIDATE_DATA_DRIFT`

Compare today, or any data, to the training data by directly comparing to the the model created in BigQuery.

In [ ]:
%%bigquery
SELECT *
FROM ML.VALIDATE_DATA_DRIFT(
    (
        SELECT *
        FROM ML.TRANSFORM(
            MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`,
            (
                SELECT *
                FROM `statmike-mlops-349915.bqml_model_monitoring.training_split`
                WHERE splits = 'TRAIN'
            )
        )
    ),
    (
        SELECT *
        FROM ML.TRANSFORM(
            MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`,
            (
                SELECT *
                FROM `statmike-mlops-349915.bqml_model_monitoring.serving`
                WHERE DATE(instance_timestamp) >= CURRENT_DATE()
            )
        )
    )
    #, MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`
);

#### Monitoring Drift (Change Over Time) With `ML.VALIDATE_DATA_DRIFT`

Compare today to 5 days ago and set the a lower, more sensitive, threshold of `0.03` (default is `0.3`) for all features.

In [ ]:
%%bigquery
SELECT *
FROM ML.VALIDATE_DATA_DRIFT(
    (
        SELECT *
        FROM ML.TRANSFORM(
            MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`,
            (
                SELECT *
                FROM `statmike-mlops-349915.bqml_model_monitoring.serving`
                WHERE DATE(instance_timestamp) = DATE_SUB(CURRENT_DATE(), INTERVAL 5 DAY)
            )
        )
    ),
    (
        SELECT *
        FROM ML.TRANSFORM(
            MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`,
            (
                SELECT *
                FROM `statmike-mlops-349915.bqml_model_monitoring.serving`
                WHERE DATE(instance_timestamp) >= CURRENT_DATE()
            )
        )
    ),
    STRUCT(
        0.03 AS categorical_default_threshold,
        0.03 AS numerical_default_threshold
    )
    #, MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`
)

---
### Monitoring Feature Attributions

BigQuery ML models, like the deep neural network classifier (`model_type = 'DNN_CLASSIFIER'`) used above, can also serve explanations using the BigQuery function [`ML.EXPLAIN_PREDICT`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-explain-predict).  This function returns feature attributions for each feature used in the model to explain how much contribution was made to the final prediction for a given instance.

**Feature Attributions**

The feature attributions from `ML.EXPLAIN_PREDICT` are [local explanations](https://cloud.google.com/bigquery/docs/xai-overview#local_versus_global_explainability), specific to the instance being predicted.  The attribution values are given for each feature and they add up to the `prediction_value`. In this case the model is a classifier and the `prediction_value` is the logit value (log-odds) of the predicted class.  

#### Serving Feature Attributions

As an example, the following cell gets the feature attributions for a single penguin with for the latest values of the features:

In [ ]:
%%bigquery
WITH
    EXPLAIN AS (
        SELECT * EXCEPT(body_mass_g, sex, island, flipper_length_mm, culmen_depth_mm, culmen_length_mm)
        FROM ML.EXPLAIN_PREDICT(
            MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`,
            (
                SELECT * EXCEPT(instance_timestamp, species)
                FROM `statmike-mlops-349915.bqml_model_monitoring.serving`
                WHERE DATE(instance_timestamp) = CURRENT_DATE()
                LIMIT 1
            ), 
            STRUCT(10 AS top_k_features)
        )
    ),
    ATTRIBUTIONS AS (
        SELECT * EXCEPT(top_feature_attributions),
        FROM EXPLAIN
        JOIN UNNEST(top_feature_attributions)
    )
SELECT * 
FROM ATTRIBUTIONS

#### Reviewing Feature Attributions

Get attributions for all penguins:

In [ ]:
%%bigquery explain
WITH
    EXPLAIN AS (
        SELECT * EXCEPT(body_mass_g, sex, island, flipper_length_mm, culmen_depth_mm, culmen_length_mm)
        FROM ML.EXPLAIN_PREDICT(
            MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`,
            (
                SELECT * EXCEPT(instance_timestamp, species)
                FROM `statmike-mlops-349915.bqml_model_monitoring.serving`
                WHERE DATE(instance_timestamp) = CURRENT_DATE()
            ), 
            STRUCT(10 AS top_k_features)
        )
    ),
    ATTRIBUTIONS AS (
        SELECT * EXCEPT(top_feature_attributions),
        FROM EXPLAIN
        JOIN UNNEST(top_feature_attributions)
    ),
    PIVOT_ATTRIBUTION AS (
        SELECT body_mass_g, sex, island, flipper_length_mm, culmen_depth_mm, culmen_length_mm
        FROM ATTRIBUTIONS PIVOT(MAX(attribution) FOR feature IN ('body_mass_g', 'sex', 'island', 'flipper_length_mm', 'culmen_depth_mm', 'culmen_length_mm'))
    )
SELECT *
FROM PIVOT_ATTRIBUTION

In [ ]:
explain.head()

#### Monitor Feature Attributions For Drift with `ML.VALIDATE_DATA_DRIFT`

Compare yesterday to 25 days ago:

In [ ]:
%%bigquery
WITH
    EXPLAIN_BASE AS (
        SELECT * EXCEPT(body_mass_g, sex, island, flipper_length_mm, culmen_depth_mm, culmen_length_mm)
        FROM ML.EXPLAIN_PREDICT(
            MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`,
            (
                SELECT * EXCEPT(instance_timestamp, species)
                FROM `statmike-mlops-349915.bqml_model_monitoring.serving`
                WHERE DATE(instance_timestamp) = DATE_SUB(CURRENT_DATE(), INTERVAL 25 DAY)
            ), 
            STRUCT(10 AS top_k_features)
        )
    ),
    ATTRIBUTIONS_BASE AS (
        SELECT * EXCEPT(top_feature_attributions),
        FROM EXPLAIN_BASE
        JOIN UNNEST(top_feature_attributions)
    ),
    PIVOT_ATTRIBUTION_BASE AS (
        SELECT body_mass_g, sex, island, flipper_length_mm, culmen_depth_mm, culmen_length_mm
        FROM ATTRIBUTIONS_BASE PIVOT(MAX(attribution) FOR feature IN ('body_mass_g', 'sex', 'island', 'flipper_length_mm', 'culmen_depth_mm', 'culmen_length_mm'))
    ),
    EXPLAIN_COMP AS (
        SELECT * EXCEPT(body_mass_g, sex, island, flipper_length_mm, culmen_depth_mm, culmen_length_mm)
        FROM ML.EXPLAIN_PREDICT(
            MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`,
            (
                SELECT * EXCEPT(instance_timestamp, species)
                FROM `statmike-mlops-349915.bqml_model_monitoring.serving`
                WHERE DATE(instance_timestamp) = DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)
            ), 
            STRUCT(10 AS top_k_features)
        )
    ),
    ATTRIBUTIONS_COMP AS (
        SELECT * EXCEPT(top_feature_attributions),
        FROM EXPLAIN_COMP
        JOIN UNNEST(top_feature_attributions)
    ),
    PIVOT_ATTRIBUTION_COMP AS (
        SELECT body_mass_g, sex, island, flipper_length_mm, culmen_depth_mm, culmen_length_mm
        FROM ATTRIBUTIONS_COMP PIVOT(MAX(attribution) FOR feature IN ('body_mass_g', 'sex', 'island', 'flipper_length_mm', 'culmen_depth_mm', 'culmen_length_mm'))
    )
SELECT *
FROM ML.VALIDATE_DATA_DRIFT(
    TABLE PIVOT_ATTRIBUTION_BASE,
    TABLE PIVOT_ATTRIBUTION_COMP
    #, MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`
);

---
## Model Monitoring For Online Inference With Vertex AI

The model trained above was trained using BigQuery ML's [`CREATE MODEL` statement](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create) along with the training option `MODEL_REGISTRY = "vertex_ai"` which automatically registers the model in the [Vertex AI Model Registry](https://cloud.google.com/vertex-ai/docs/model-registry/introduction).  This means the BigQuery ML models can be managed directly in Vertex AI - [more information](https://cloud.google.com/bigquery/docs/managing-models-vertex).

Models in the Vertex AI Model Registry can be used to [serve batch or online predictions](https://cloud.google.com/vertex-ai/docs/predictions/get-predictions).  In this section the [Vertex AI Python SDK](https://cloud.google.com/vertex-ai/docs/start/use-vertex-ai-python-sdk) is used to [deploy the model](https://cloud.google.com/vertex-ai/docs/general/deployment) created in this workflow and [get online predictions](https://cloud.google.com/vertex-ai/docs/predictions/get-online-predictions).

The deployment of the model will include an extra setting to [setup logging of prediction request and responses to BigQuery](https://cloud.google.com/vertex-ai/docs/predictions/online-prediction-logging#enabling-and-disabling).  By logging the request and responses to BigQuery we can easily use the model monitoring functions covered in this tutorial.

### Setup Client

In [ ]:
from google.cloud import aiplatform
aiplatform.init(project = 'statmike-mlops-349915', location = 'us-central1')

### Get Model From Vertex AI Model Registry

In [ ]:
model = aiplatform.Model(model_name = 'classify_species_rf')
model.versioned_resource_name

### Deploy Model To Endpoint

In [ ]:
endpoint = model.deploy()

In [ ]:
endpoint.resource_name

### Get Data For Predictions

Get a sample of data from the serving table to use as online prediction requests.

In [ ]:
%%bigquery online
SELECT * EXCEPT(species, instance_timestamp)
FROM statmike-mlops-349915.bqml_model_monitoring.serving
WHERE RAND() < 0.1

In [ ]:
import numpy as np
online = online.dropna()
online['sex'].replace(np.nan, '', inplace=True)
online = online.to_dict(orient = 'records')
online[0]

### Get Predictions

In [ ]:
endpoint.predict(instances = online[0:1])

### Enable Prediction request-response logging To BigQuery

The endpoint can be enabled for logging both the request and response for predictions.  These logs are directed to a BigQuery table.  This is enable throught the `gcloud ai endpoints update` command - [reference](https://cloud.google.com/sdk/gcloud/reference/ai/endpoints/update).

In [ ]:
endpoint.name

In [ ]:
!gcloud ai endpoints update $endpoint.name \
--region=us-central1 \
--request-response-logging-table=bq://statmike-mlops-349915.bqml_model_monitoring.serving_online

### Send Prediction Requests To Endpoint

These will get logged to the BigQuery table specified above.

In [ ]:
for instance in online:
    prediction = endpoint.predict(instances = [instance])

In [ ]:
%%bigquery
SELECT *
FROM `statmike-mlops-349915.bqml_model_monitoring.serving_online`
LIMIT 5

### Create A View Of All Serving Data

Combine the batch serving data with the online serving data in a view.

In [ ]:
%%bigquery
SELECT
    logging_time as instance_timestamp,
    JSON_VALUE(request, "$.island") as island,
    JSON_VALUE(request, "$.sex") as sex,
    CAST(JSON_VALUE(request, "$.flipper_length_mm") AS FLOAT64) as flipper_length_mm,
    CAST(JSON_VALUE(request, "$.body_mass_g") AS FLOAT64) as body_mass_g,
    CAST(JSON_VALUE(request, "$.culmen_length_mm") AS FLOAT64) as culmen_length_mm,
    CAST(JSON_VALUE(request, "$.culmen_depth_mm") AS FLOAT64) as culmen_depth_mm
FROM `statmike-mlops-349915.bqml_model_monitoring.serving_online`,
UNNEST(request_payload) as request
LIMIT 5

In [ ]:
%%bigquery
CREATE OR REPLACE VIEW `statmike-mlops-349915.bqml_model_monitoring.serving_all` AS
WITH
    BATCH AS (
        SELECT instance_timestamp, island, sex, flipper_length_mm, body_mass_g, culmen_length_mm, culmen_depth_mm
        FROM `statmike-mlops-349915.bqml_model_monitoring.serving`
    ),
    ONLINE AS (
        SELECT
            logging_time as instance_timestamp,
            JSON_VALUE(request, "$.island") as island,
            JSON_VALUE(request, "$.sex") as sex,
            CAST(JSON_VALUE(request, "$.flipper_length_mm") AS FLOAT64) as flipper_length_mm,
            CAST(JSON_VALUE(request, "$.body_mass_g") AS FLOAT64) as body_mass_g,
            CAST(JSON_VALUE(request, "$.culmen_length_mm") AS FLOAT64) as culmen_length_mm,
            CAST(JSON_VALUE(request, "$.culmen_depth_mm") AS FLOAT64) as culmen_depth_mm
        FROM `statmike-mlops-349915.bqml_model_monitoring.serving_online`,
        UNNEST(request_payload) as request
    )
SELECT * FROM BATCH
UNION ALL
SELECT * FROM ONLINE


### Monitoring Skew (Change from training) With `ML.VALIDATE_DATA_SKEW`

Compare today, or any data, to the training data by directly comparing to the the model created in BigQuery.

In [ ]:
%%bigquery c_skew
SELECT *
FROM ML.VALIDATE_DATA_SKEW(
    MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`,
    (
        SELECT *
        FROM `statmike-mlops-349915.bqml_model_monitoring.serving_all`
        WHERE DATE(instance_timestamp) >= CURRENT_DATE()
    )
    ,STRUCT(TRUE AS enable_visualization_link)
);

In [ ]:
c_skew

In [ ]:
display(HTML(f'<a href="{c_skew.visualization_link[0]}" target="_blank" style="font-size: 18px;">Open Google Cloud Console in a new tab</a>'))

### Monitoring Drift (Change Over Time) With `ML.VALIDATE_DATA_DRIFT`

Compare today to 5 days ago and set the a lower, more sensitive, threshold of `0.03` (default is `0.3`) for all features.

In [ ]:
%%bigquery c_drift
SELECT *
FROM ML.VALIDATE_DATA_DRIFT(
    (
        SELECT * EXCEPT(instance_timestamp)
        FROM `statmike-mlops-349915.bqml_model_monitoring.serving_all`
        WHERE DATE(instance_timestamp) = DATE_SUB(CURRENT_DATE(), INTERVAL 5 DAY)
    ),
    (
        SELECT * EXCEPT(instance_timestamp)
        FROM `statmike-mlops-349915.bqml_model_monitoring.serving_all`
        WHERE DATE(instance_timestamp) = CURRENT_DATE() 
    ),
    STRUCT(
        0.03 AS categorical_default_threshold,
        0.03 AS numerical_default_threshold
    )
    , MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_logistic`
);

In [ ]:
c_drift

In [ ]:
display(HTML(f'<a href="{c_drift.visualization_link[0]}" target="_blank" style="font-size: 18px;">Open Google Cloud Console in a new tab</a>'))

### Remove Endpoint 
To prevent ongoing cost for this demonstration, remove the endpoint.  

In [ ]:
endpoint.delete(force = True)

---
## Continous Monitoring

The methods above can be combined into a job that runs both skew and drift detection.  The jobs can be scheduled or triggered to make the process of model monitoring continous - continous monitoring.  If the model quality can be evaluated with confidence then the automation can even be extended to retrain and update the model for continous training.   

This section shows one of many ways to create a model monitoring job. For models created in BigQuery with BigQuery ML it also shows a retraining action as part of the job when alerts are detected.

These types of jobs could be automated for continous monitoring with one of these methods:
- [BigQuery Scheduled Queries](https://cloud.google.com/bigquery/docs/scheduling-queries)
    - Featured below
- [Dataform](https://cloud.google.com/dataform/docs/quickstart-create-workflow)
- [Workflows](https://cloud.google.com/workflows/docs/tutorials/bigquery-parallel-jobs)
- [Cloud Composer](https://cloud.google.com/composer/docs/data-analytics-googlecloud)
- [Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/gcpc-list#bqml_components)

This section will show how to:
- Develop model monitoring jobs:
    - with alterting and retraining
- Build tables to capture continous monitoring results
- Extend monitoring jobs to BigQuery Scheduled Queries
- Backfill monitoring jobs for a past time period

### Model Monitoring Job: With Alerting And Retraining

A BigQuery SQL job that uses the [procedural language](https://cloud.google.com/bigquery/docs/reference/standard-sql/procedural-language) to execute multiple statements (steps) as a single query.  In this case, monitoring skew and drift, detecting any feature level anomalies in either and reporting back the anomalies via a log note with a forced error.

This job also includes retraining of the BigQuery ML model:
In this case the example is simplified to retrain the model and directly implement it as a replacement with `CREATE OR REPLACE MODEL`.  In a production enviornment additional pre/post steps might take place as part of this workflow:
- on skew/drift detection:
    - Prior to retraining:
        - ensure detection is not due to data errors, schema changes, or system changes
    - After retrainning:
        - evaluate the model and compare to current production version
        - conditionally deploy the new version if it is better in any/all key metrics
        - consider rollout strategies for the retrained model
        - trigger human review before deployment of the replacement

In [ ]:
%%bigquery
DECLARE drift_anomalies ARRAY<STRUCT<input STRING, visualization_link STRING>>;
DECLARE drift STRING;
DECLARE skew_anomalies ARRAY<STRUCT<input STRING, visualization_link STRING>>;
DECLARE skew STRING;

# Monitor Drift:
SET drift_anomalies = (
    SELECT ARRAY_AGG(STRUCT(input, visualization_link))
    FROM ML.VALIDATE_DATA_DRIFT(
        (# base
            SELECT * EXCEPT(species, instance_timestamp)
            FROM `statmike-mlops-349915.bqml_model_monitoring.serving`
            WHERE DATE(instance_timestamp) = DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY)
        ),
        (# compare
            SELECT * EXCEPT(species, instance_timestamp)
            FROM `statmike-mlops-349915.bqml_model_monitoring.serving`
            WHERE DATE(instance_timestamp) = DATE_SUB(CURRENT_DATE(), INTERVAL 29 DAY)
        ),
        STRUCT(
            0.4 AS categorical_default_threshold,
            0.4 AS numerical_default_threshold
        )
        , MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_logistic`
    )
    WHERE is_anomaly = True
);
IF(ARRAY_LENGTH(drift_anomalies) > 0) THEN
    SET drift = CONCAT(
        "\n\tDrift: detected in the following features",
        (
            SELECT STRING_AGG(
                CONCAT(
                    '\n\t\t',
                    da.input,
                    ': ',
                    da.visualization_link
                )
            )
            FROM UNNEST(drift_anomalies) as da
        )
    );
    ELSE SET drift = '\n\tDrift: not detected.';
END IF;

# Monitor Skew
SET skew_anomalies = (
    SELECT ARRAY_AGG(STRUCT(input, visualization_link))
    FROM ML.VALIDATE_DATA_SKEW(
        # base
        MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`,
        (# compare
            SELECT * EXCEPT(species, instance_timestamp)
            FROM `statmike-mlops-349915.bqml_model_monitoring.serving`
            WHERE DATE(instance_timestamp) = DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY)
        )
        ,STRUCT(TRUE AS enable_visualization_link)
    )
    WHERE is_anomaly = True
);
IF(ARRAY_LENGTH(skew_anomalies) > 0) THEN
    SET skew = CONCAT(
        "\n\tSkew: detected in the following features",
        (
            SELECT STRING_AGG(
                CONCAT(
                    '\n\t\t',
                    sa.input,
                    ': ',
                    sa.visualization_link
                )
            )
            FROM UNNEST(skew_anomalies) as sa
        )
    );
    ELSE SET skew = '\n\tSkew: not detected.';
END IF;

# Prepare Alert
IF(ARRAY_LENGTH(drift_anomalies) > 0 OR ARRAY_LENGTH(skew_anomalies) > 0) THEN
    BEGIN
        DECLARE train_accuracy FLOAT64;
        DECLARE recent_accuracy FLOAT64;
        DECLARE retrain_accuracy FLOAT64;

        # get current models evaluation metrics: accuracy
        SET train_accuracy = (
            SELECT accuracy
            FROM ML.EVALUATE (
                MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`
            )
        );

        # get current models recent evaluation metrics: accuracy
        SET recent_accuracy = (
            SELECT accuracy
            FROM ML.EVALUATE (
                MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`,
                (
                    SELECT *
                    FROM `statmike-mlops-349915.bqml_model_monitoring.serving`
                    WHERE DATE(instance_timestamp) = DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY) 
                )
            )
        );

        # retrain the model
        CREATE OR REPLACE MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`
            TRANSFORM(
                ML.ROBUST_SCALER(body_mass_g) OVER() AS body_mass_g,
                ML.STANDARD_SCALER(culmen_length_mm) OVER() AS culmen_length_mm,
                ML.STANDARD_SCALER(culmen_depth_mm) OVER() AS culmen_depth_mm,
                ML.QUANTILE_BUCKETIZE(flipper_length_mm, 3) OVER() AS flipper_length_mm,
                ML.IMPUTER(sex, 'most_frequent') OVER() AS sex,
                ML.IMPUTER(island, 'most_frequent') OVER() AS island,
                species
            )
            OPTIONS(
                MODEL_TYPE = 'RANDOM_FOREST_CLASSIFIER',
                INPUT_LABEL_COLS = ['species'],

                # data specifics
                DATA_SPLIT_METHOD = 'AUTO_SPLIT',

                # model specifics
                AUTO_CLASS_WEIGHTS = FALSE,
                NUM_PARALLEL_TREE= 150,
                TREE_METHOD = 'HIST',
                SUBSAMPLE = 0.85,
                COLSAMPLE_BYTREE = 0.9,
                ENABLE_GLOBAL_EXPLAIN = TRUE,

                # register model in Vertex AI For Online Serving
                MODEL_REGISTRY = 'VERTEX_AI'
            )
        AS
            SELECT species, island, culmen_length_mm, culmen_depth_mm, sex, flipper_length_mm, body_mass_g
            FROM `statmike-mlops-349915.bqml_model_monitoring.training`
            UNION ALL
            SELECT species, island, culmen_length_mm, culmen_depth_mm, sex, flipper_length_mm, body_mass_g
            FROM `statmike-mlops-349915.bqml_model_monitoring.serving`
            WHERE DATE(instance_timestamp) = DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY)
        ;

        # get retrained models evaluation metrics: accuracy
        SET retrain_accuracy = (
            SELECT accuracy
            FROM ML.EVALUATE (
                MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`
            )
        );

        SELECT ERROR(
            CONCAT(
                "\n\nMonitoring Report:",
                drift,
                skew,
                "\n\nThe Model was retrained:",
                "\n\taccuracy of prior model: ", train_accuracy,
                "\n\trecent accuracy of prior model: ", recent_accuracy,
                "\n\taccuracy after retraining: ", retrain_accuracy,
                "\n\n"
            )
        );
    END;
    ELSE
        SELECT ERROR(
            CONCAT(
                "\n\nMonitoring Report:",
                drift,
                skew,
                "\n\n"
            )
        );
END IF;

### From Job To Scheduled Query

The model monitoring job created above can be modified and scheduled using [scheduled queries](https://cloud.google.com/bigquery/docs/scheduling-queries).  A key feature of the modification is making use of the parameter `@run_date` to parameterize the queries reference to dates (see [configuration options](https://cloud.google.com/bigquery/docs/scheduling-queries#configuration_options)).  This allows the query to run as though it is on different dates which will be helpful as we backfill the model monitoring runs after scheduling for today and forward.

Note, the next cell is writing the content of the query to a file named `model_monitoring_job.sql`.
Other modification to the job above:
- Drift is set to compare the last weeks serving data to the prior week
- Skew is set to compare the last weeks serving data to the baseline training data

In [ ]:
%%writefile model_monitoring_job.sql
DECLARE drift_anomalies ARRAY<STRUCT<input STRING, visualization_link STRING>>;
DECLARE drift STRING;
DECLARE skew_anomalies ARRAY<STRUCT<input STRING, visualization_link STRING>>;
DECLARE skew STRING;

# Monitor Drift:
SET drift_anomalies = (
    SELECT ARRAY_AGG(STRUCT(input, visualization_link))
    FROM ML.VALIDATE_DATA_DRIFT(
        (# base
            SELECT * EXCEPT(instance_timestamp)
            FROM `statmike-mlops-349915.bqml_model_monitoring.serving_all`
            WHERE
                DATE(instance_timestamp) >= DATE_SUB(@run_date, INTERVAL 2 WEEK)
                AND
                DATE(instance_timestamp) < DATE_SUB(@run_date, INTERVAL 1 WEEK)
        ),
        (# compare
            SELECT * EXCEPT(instance_timestamp)
            FROM `statmike-mlops-349915.bqml_model_monitoring.serving_all`
            WHERE DATE(instance_timestamp) >= DATE_SUB(@run_date, INTERVAL 1 WEEK)
        ),
        STRUCT(
            0.4 AS categorical_default_threshold,
            0.4 AS numerical_default_threshold
        )
        , MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_logistic`
    )
    WHERE is_anomaly = True
);
IF(ARRAY_LENGTH(drift_anomalies) > 0) THEN
    SET drift = CONCAT(
        "\n\tDrift: detected in the following features",
        (
            SELECT STRING_AGG(
                CONCAT(
                    '\n\t\t',
                    da.input,
                    ': ',
                    da.visualization_link
                )
            )
            FROM UNNEST(drift_anomalies) as da
        )
    );
    ELSE SET drift = '\n\tDrift: not detected.';
END IF;

# Monitor Skew
SET skew_anomalies = (
    SELECT ARRAY_AGG(STRUCT(input, visualization_link))
    FROM ML.VALIDATE_DATA_SKEW(
        # base
        MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`,
        (# compare
            SELECT * EXCEPT(instance_timestamp)
            FROM `statmike-mlops-349915.bqml_model_monitoring.serving_all`
            WHERE DATE(instance_timestamp) >= DATE_SUB(@run_date, INTERVAL 1 WEEK)
        )
        ,STRUCT(TRUE AS enable_visualization_link)
    )
    WHERE is_anomaly = True
);
IF(ARRAY_LENGTH(skew_anomalies) > 0) THEN
    SET skew = CONCAT(
        "\n\tSkew: detected in the following features",
        (
            SELECT STRING_AGG(
                CONCAT(
                    '\n\t\t',
                    sa.input,
                    ': ',
                    sa.visualization_link
                )
            )
            FROM UNNEST(skew_anomalies) as sa
        )
    );
    ELSE SET skew = '\n\tSkew: not detected.';
END IF;

# Prepare Alert
IF(ARRAY_LENGTH(drift_anomalies) > 0 OR ARRAY_LENGTH(skew_anomalies) > 0) THEN
    BEGIN
        DECLARE train_accuracy FLOAT64;
        DECLARE recent_accuracy FLOAT64;
        DECLARE retrain_accuracy FLOAT64;

        # get current models evaluation metrics: accuracy
        SET train_accuracy = (
            SELECT accuracy
            FROM ML.EVALUATE (
                MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`
            )
        );

        # get current models recent evaluation metrics: accuracy
        SET recent_accuracy = (
            SELECT accuracy
            FROM ML.EVALUATE (
                MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`,
                (
                    SELECT *
                    FROM `statmike-mlops-349915.bqml_model_monitoring.serving`
                    WHERE DATE(instance_timestamp) >= DATE_SUB(@run_date, INTERVAL 1 WEEK) 
                )
            )
        );

        # retrain the model
        CREATE OR REPLACE MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`
            TRANSFORM(
                ML.ROBUST_SCALER(body_mass_g) OVER() AS body_mass_g,
                ML.STANDARD_SCALER(culmen_length_mm) OVER() AS culmen_length_mm,
                ML.STANDARD_SCALER(culmen_depth_mm) OVER() AS culmen_depth_mm,
                ML.QUANTILE_BUCKETIZE(flipper_length_mm, 3) OVER() AS flipper_length_mm,
                ML.IMPUTER(sex, 'most_frequent') OVER() AS sex,
                ML.IMPUTER(island, 'most_frequent') OVER() AS island,
                species
            )
            OPTIONS(
                MODEL_TYPE = 'RANDOM_FOREST_CLASSIFIER',
                INPUT_LABEL_COLS = ['species'],

                # data specifics
                DATA_SPLIT_METHOD = 'AUTO_SPLIT',

                # model specifics
                AUTO_CLASS_WEIGHTS = FALSE,
                NUM_PARALLEL_TREE= 150,
                TREE_METHOD = 'HIST',
                SUBSAMPLE = 0.85,
                COLSAMPLE_BYTREE = 0.9,
                ENABLE_GLOBAL_EXPLAIN = TRUE,

                # register model in Vertex AI For Online Serving
                MODEL_REGISTRY = 'VERTEX_AI',
                VERTEX_AI_MODEL_ID = 'classify_species_rf'
            )
        AS
            SELECT species, island, culmen_length_mm, culmen_depth_mm, sex, flipper_length_mm, body_mass_g
            FROM `statmike-mlops-349915.bqml_model_monitoring.training_split`
            WHERE splits = 'TRAIN'
            UNION ALL
            SELECT species, island, culmen_length_mm, culmen_depth_mm, sex, flipper_length_mm, body_mass_g
            FROM `statmike-mlops-349915.bqml_model_monitoring.serving`
            WHERE DATE(instance_timestamp) < @run_date
        ;

        # get retrained models evaluation metrics: accuracy
        SET retrain_accuracy = (
            SELECT accuracy
            FROM ML.EVALUATE (
                MODEL `statmike-mlops-349915.bqml_model_monitoring.classify_species_rf`
            )
        );

        SELECT ERROR(
            CONCAT(
                "\n\nMonitoring Report for ", @run_date, ":",
                "\n\t", drift,
                "\n\t", skew,
                "\n\nThe Model was retrained:",
                "\n\taccuracy of prior model: ", train_accuracy,
                "\n\trecent accuracy of prior model: ", recent_accuracy,
                "\n\taccuracy after retraining: ", retrain_accuracy,
                "\n\n"
            )
        );
    END;
    # ELSE ... supressed alerts on runs without anomalies detected
END IF;

### Schedule The Query

The query can be scheduled in [a number of ways](https://cloud.google.com/bigquery/docs/scheduling-queries#set_up_scheduled_queries) including the console, bq cli, and Python. Here, the Python client will be used.
- **References**
    - [Python Client For BigQuery Data Transfer](https://cloud.google.com/python/docs/reference/bigquerydatatransfer/latest)
    - [Set up schedule queries with Python client](https://cloud.google.com/bigquery/docs/scheduling-queries#python)

In [ ]:
from google.cloud import bigquery_datatransfer
from datetime import datetime, timezone, timedelta
import time
transfer_client = bigquery_datatransfer.DataTransferServiceClient()

In [ ]:
start_time = datetime.now(timezone.utc) + timedelta(days=1) # start tomorrow
start_time

In [ ]:
with open('model_monitoring_job.sql', 'r') as file:
    query = file.read()

In [ ]:
transfer_client.common_location_path('statmike-mlops-349915', 'us')

In [ ]:
scheduled_query = transfer_client.create_transfer_config(
    #parent = transfer_client.common_project_path('statmike-mlops-349915'),
    parent = transfer_client.common_location_path('statmike-mlops-349915', 'us'),
    transfer_config = bigquery_datatransfer.TransferConfig(
        display_name = 'Continous Model Monitoring',
        data_source_id = 'scheduled_query',
        params = {'query': query},
        schedule = 'every 24 hours',
        schedule_options = bigquery_datatransfer.ScheduleOptions(
            start_time=start_time.strftime("%Y-%m-%dT%H:%M:%SZ")
        ),
        email_preferences = bigquery_datatransfer.EmailPreferences(
            enable_failure_email = True
        )
    )
)

In [ ]:
scheduled_query.display_name, scheduled_query.name

In [ ]:
scheduled_query.next_run_time.strftime("%Y-%m-%d %H:%M:%S.%f")

Take a moment to see the query in the console:
- Console > BigQuery > Scheduled Queries

### Backfill History With Scheduled Query

While the client can use a start and end time to run a range of backfill dates, it will runs these backwards from most recent to the earliest time.  Due to the nature of the model monitoring job it is more desirable to run these in the order from earliest to most recent.  This is accomplished by launching each days job separately in the desired order.

In [ ]:
# -30 days
start_time = datetime.now(timezone.utc) + timedelta(days = -30)
backfill_job = transfer_client.start_manual_transfer_runs(
    request = bigquery_datatransfer.StartManualTransferRunsRequest(
        parent = scheduled_query.name,
        requested_run_time = start_time 
    )
)

In [ ]:
backfill_job.runs[0].name

In [ ]:
transfer_client.get_transfer_run(
    name = backfill_job.runs[0].name
).state.name

**NOTE:** A State of 'FAILED' is actually the correct end state as the method of using SQL ERRORS to pass the results and trigger notification emails was used. A State of 'SUCCEEDED' will represent runs without an anomaly detected.  [Read more about TrasferState](https://cloud.google.com/python/docs/reference/bigquerydatatransfer/latest/google.cloud.bigquery_datatransfer_v1.types.TransferState).

In [ ]:
while True:
    state = transfer_client.get_transfer_run(
        name = backfill_job.runs[0].name
    ).state.name
    if state in ['FAILED', 'SUCCEEDED', 'CANCELLED']:
        break
    else:
        time.sleep(30)

In [ ]:
backfill_jobs =[backfill_job]

In [ ]:
# Loop From -29 to -0 days
for days in range(30)[::-1]:
    start_time = datetime.now(timezone.utc) + timedelta(days = -1*days)
    print(start_time)
    # start job
    backfill_jobs.append(
        transfer_client.start_manual_transfer_runs(
            request = bigquery_datatransfer.StartManualTransferRunsRequest(
                parent = scheduled_query.name,
                requested_run_time = start_time 
            )
        )
    )
    # monitor job before continuing:
    while True:
        state = transfer_client.get_transfer_run(
            name = backfill_jobs[-1].runs[0].name
        ).state.name
        if state in ['FAILED', 'SUCCEEDED', 'CANCELLED']:
            break
        else:
            time.sleep(10)    

In [ ]:
backfill_results = [transfer_client.get_transfer_run(name = j.runs[0].name) for j in backfill_jobs]

In [ ]:
pd.DataFrame(
    {
        'Date':[r.run_time for r in backfill_results],
        'Result':[r.error_status.message for r in backfill_results]
    }
).style.set_properties(**{'text-align': 'left', 'white-space': 'pre-wrap'})

---
## More Resources

Did you also check out the ['BigQuery ML (BQML) - Model Monitoring Introduction'](./bqml-model-monitoring-introduction.ipynb) for quick overview of model monitoring with BigQuery?  If not, try it out also. 

Read more about model monitoring in the documentation for the model monitoring functions:
* [BigQuery ML Documentation](https://cloud.google.com/bigquery-ml/docs/)
* [BigQuery ML Model Monitoring](https://cloud.google.com/bigquery/docs/model-monitoring-overview)
    - [`ML.DESCRIBE_DATA`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-describe-data)
    - [`ML.VALIDATE_DATA_SKEW`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-validate-data-skew)
    - [`ML.VALIDATE_DATA_DRIFT`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-validate-data-drift)
    - [`ML.TFDV_DESCRIBE`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-tfdv-describe)
    - [`ML.TFDV_VALIDATE`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-tfdv-validate)

---
## Remove Resources Created In This Notebook

The content is contained within the BigQuery dataset named `bqml_model_monitoring` created above and Vertex AI (endpoint created above).

The following will delete the Vertex AI and BigQuery resources created above and the BigQuery dataset if it is empty.

In [ ]:
# to remove resources change this flag to True:
remove = False

In [ ]:
if remove:
    # client for BigQuery
    from google.cloud import bigquery
    bq = bigquery.Client(project = 'statmike-mlops-349915')
    
    # delete BigQuery table(s) and model(s) created above
    bq.delete_table('statmike-mlops-349915.bqml_model_monitoring.training_split')
    bq.delete_table('statmike-mlops-349915.bqml_model_monitoring.serving')
    bq.delete_table('statmike-mlops-349915.bqml_model_monitoring.serving_online')
    bq.delete_table('statmike-mlops-349915.bqml_model_monitoring.serving_all')
    bq.delete_model('statmike-mlops-349915.bqml_model_monitoring.classify_species_rf')
    
    # delete BigQuery dataset - fails if other content was in the dataset than what this notebook created
    bq.delete_dataset('statmike-mlops-349915.bqml_model_monitoring', delete_contents = False)
    
    # delete the scheduled query
    transfer_client.delete_transfer_config(
        name = scheduled_query.name
    )